Motivation: Understand the difference in how people approach dating at different age groups, based on advice posts in two subreddits focused on different age groups:
* [r/dating](https://www.reddit.com/r/dating)
* [r/datingoverthirty](https://www.reddit.com/r/datingoverthirty)

In [93]:
import praw
import pandas as pd

In [94]:
reddit = praw.Reddit(client_id='Io2p8TWnTj6hlgrO62G_jQ',
    client_secret='5cD0nzDtEF3o8316kewDQ5Y0A0Blwg',
    user_agent='Post Description Extraction',
    username='sinkorfloat',
    password='amidships'
)

In [95]:
# Get the top comment each submission, sorted by upvote (as seen on the site)

def get_top_comment(submission_id):
    post = reddit.submission(id = submission_id)
    post.comment_sort = 'top' # cite: https://www.reddit.com/r/redditdev/comments/mzloap/extract_only_the_top_comment_for_a_submission/gw1h38f/
    top_comments = post.comments[0:5]

    #Some subreddits include automoderator sticky posts at the top of each thread (keep it civil, see the rules...).  Filtering these out
    non_mod_comments = [comment.body for comment in top_comments if comment.stickied == False]
    if len(non_mod_comments) ==0:
        top_comment = ''
    else: 
        top_comment = non_mod_comments[0]
    return top_comment

### Content Extraction

In [96]:
#Extract 2000 recent submissions
subreddit_1 = reddit.subreddit('dating') #https://praw.readthedocs.io/en/stable/code_overview/reddit_instance.html
posts1 = subreddit_1.new(limit = 2000)

subreddit_2 = reddit.subreddit('datingoverthirty')
posts2 = subreddit_2.new(limit = 2000)

In [97]:
# Create a dataframe of all posts, including selftext, top_commment_text
data = []
ids = []

for post in posts1:
    if post.id not in ids:
        urls.append([post.id])      
            
        data.append([post.subreddit, post.id, post.created_utc, post.title, post.selftext, get_top_comment(post.id)])
    else:
        continue
        
for post in posts2:
    if post.url not in urls:
        urls.append([post.id])
        post.comment_sort = 'top'
        data.append([post.subreddit, post.id, post.created_utc, post.title, post.selftext, get_top_comment(post.id)])
    else:
        continue

posts_df = pd.DataFrame(data, columns = ['subreddit', 'id', 'created_utc', 'title', 'selftext', 'top_comment_text'])

In [100]:
posts_df.shape

(1971, 6)

In [103]:
posts_df['subreddit'].value_counts()

datingoverthirty    998
dating              973
Name: subreddit, dtype: int64

In [107]:
posts_df.head(5)

,subreddit,id,created_utc,title,selftext,top_comment_text
0,dating,1471ube,1.686509e+09,Am I Clueless?,So there is this girl I’ve known my whole life...,
1,dating,1471t0w,1.686509e+09,Is my Ex talking to the girl I like on purpose?,The title explains it all but here's a bit of ...,
2,dating,1471gsn,1.686509e+09,Advice for Dealing with Anxious Attachment Sty...,I suffer from Anxious Attachment Style. It ve...,
3,dating,1471coz,1.686508e+09,I want to send a letter to a guy I dated a few...,Ages at the time: me (26) him (29)\n\nA few ye...,
4,dating,14716mt,1.686508e+09,A bit mopey,I (22m) have a date tonight (that I can't post...,


In [108]:
pd.to_datetime(posts_df['created_utc']).head()

0   1970-01-01 00:00:01.686509373
1   1970-01-01 00:00:01.686509291
2   1970-01-01 00:00:01.686508506
3   1970-01-01 00:00:01.686508235
4   1970-01-01 00:00:01.686507839
Name: created_utc, dtype: datetime64[ns]

In [ ]:
from datetime import datetime
now = datetime.now().strftime("%Y-%m-%d %H:%M")
posts_df.to_csv(f'data/reddit_posts_raw_{now}.csv', index = False)